In [5]:
!pip install selenium
!pip install selenium chromedriver-autoinstaller
!pip install webdriver-manager
!pip install undetected-chromedriver

In [2]:
# !apt-get update
# !apt-get install -y chromium-browser
# !apt-get install chromium-chromedriver

In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from PIL import Image
from io import BytesIO
import os
import io
import shutil

import pandas as pd
import re
import requests

from time import sleep
import time

In [13]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os

# Get the current working directory
current_dir = os.getcwd()
# Define download directory within the Gemini folder
download_dir = os.path.join(current_dir, 'Gemini')
if not os.path.exists(download_dir):
    os.makedirs(download_dir)
    
# Define Chrome options and experimental preferences
options = uc.ChromeOptions()
prefs = {'download.default_directory': download_dir,
         'profile.default_content_setting_values.automatic_downloads': 1}
options.add_experimental_option("prefs", prefs)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Initialize undetected Chrome driver with the configured options
driver = uc.Chrome(options=options)

# Load the website
driver.get("https://gemini.google.com/app/0fd678f1e351de31")

# Wait for the first sign-in button to be clickable and click it
wait = WebDriverWait(driver, 20)

In [14]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import os

# Read prompts from prompts.txt file
with open('prompts.txt', 'r') as file:
    prompts = file.readlines()

# Switch to the default content
driver.switch_to.default_content()

# Find the div text box element using its inspect element
div_text_box = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.CSS_SELECTOR, 'div.ql-editor'))
)

# Initialize variables to track the latest downloaded image
latest_image_id = None
latest_download_time = time.time()

# Loop through each prompt and generate images
for prompt_number, prompt_text in enumerate(prompts, start=1):
    full_prompt_text = f"Create an image of: {prompt_text.strip()}"
    # Enter your text into the div text box
    div_text_box.clear()  # Clear any existing text
    div_text_box.send_keys(full_prompt_text)
    # Press Enter to submit the text
    div_text_box.send_keys(Keys.ENTER)

    # Wait for the image to start generating
    time.sleep(2)  # Adjust this time based on how long it typically takes to start generating an image

    # Check if a new image has been generated since the last download
    image_found = False
    while not image_found:
        try:
            # Find the image container
            image_container = driver.find_element(By.CSS_SELECTOR, 'div.image-container')
            image_id = image_container.get_attribute('data-image-id')

            # Check if this is a new image that hasn't been downloaded yet
            if image_id != latest_image_id or time.time() - latest_download_time > 60:
                # Wait for the image to be fully loaded (assuming an img element appears inside the image container)
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.image-container img')))
                # Find all download buttons on the page
                download_buttons = driver.find_elements(By.CSS_SELECTOR, 'div.button-icon-wrapper mat-icon[data-mat-icon-type="font"]')
                # Click on the last download button only if multiple downloads are allowed
                if download_buttons and len(download_buttons) > 1:
                    latest_download_button = download_buttons[-1]
                    # Scroll to the download button using JavaScript
                    driver.execute_script("arguments[0].scrollIntoView(true);", latest_download_button)
                    # Click the download button using JavaScript
                    driver.execute_script("arguments[0].click();", latest_download_button)
                    image_found = True  # Image found and downloaded
                    latest_image_id = image_id  # Update the latest image ID
                    latest_download_time = time.time()  # Update the latest download time
            else:
                time.sleep(1)  # Wait before checking again for the next image
        except:
            # Handle exceptions, such as if the image container or download button is not found
            time.sleep(1)  # Wait before retrying to find the image

    # Wait for a moment to ensure the download is complete
    time.sleep(5)  # Adjust this time based on how long it typically takes to download an image

    # Rename the downloaded file as "Tweet_<prompt_number>.jpeg"
    downloaded_files = os.listdir(download_dir)
    for file_name in downloaded_files:
        if file_name.endswith('.jpeg'):
            new_file_name = f"Tweet_{prompt_number}.jpeg"
            os.rename(os.path.join(download_dir, file_name), os.path.join(download_dir, new_file_name))
            break  # Exit loop after renaming the first downloaded file

In [15]:
driver.quit()